In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer2 import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
from pathlib import Path
import os
import random

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine, Sphere, Rgba, CspaceFreeRegion
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)




pendulum_world = FindResourceOrThrow("drake/C_Iris_Examples/assets/single_pendulum_world.urdf")

models = []
models.append(parser.AddModelFromFile(pendulum_world))

plant.Finalize()
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(1)

# #compute limits in t-space
limits_q = [plant.GetPositionLowerLimits().item(), plant.GetPositionUpperLimits().item()]
limits_s = [Ratfk.ComputeTValue(np.array([q]), q_star) for q in limits_q]
do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7006
INFO:drake:Meshcat listening for connections at http://localhost:7007


In [6]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)

id_pairs_of_interest = [(gid1, gid2) for i, gid1 in enumerate(gids) for gid2 in gids[i+1:] ]
print(id_pairs_of_interest)
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


[(<GeometryId value=38>, <GeometryId value=28>), (<GeometryId value=38>, <GeometryId value=25>), (<GeometryId value=28>, <GeometryId value=25>)]


In [7]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))


## Set up the sliders so we can move the plant around manually

In [9]:
sliders = []
sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits().item(),
                                   max=plant.GetPositionUpperLimits().item(), 
                                   value=0, description='q0'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.array([0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.57, min=-1.57)

In [10]:
# import time
# for q in np.linspace(plant.GetPositionLowerLimits().item(),
#                     plant.GetPositionUpperLimits().item(), 100):
#     visualizer.showres(np.array([q]))
#     time.sleep(0.1)

In [11]:
s0 = np.array([-0.0])
s1 = np.array([limits_s[1] - 0.1])
def plot_edge(s0, s1):
    visualizer.meshcat2.SetObject("/s0",
                                 Sphere(0.05),
                                 Rgba(0.0, 1.0, 0, 1))
    s0_3d = np.array([s0.item(), 0, 0])
    s1_3d = np.array([s1.item(), 0, 0])
    visualizer.meshcat2.SetTransform("/s0",
                                  RigidTransform(RotationMatrix(),
                                                 s0_3d))
    
    visualizer.meshcat2.SetObject("/s1",
                                 Sphere(0.05),
                                 Rgba(1.0, 0.0, 0, 1))
    visualizer.meshcat2.SetTransform("/s1",
                                   RigidTransform(RotationMatrix(),
                                                 s1_3d))
    print(np.hstack([s0_3d[:,np.newaxis], s1_3d[:, np.newaxis]]))
    visualizer.meshcat2.SetLine("/edge", 
                                np.hstack([s0_3d[:,np.newaxis], s1_3d[:, np.newaxis]]),
                                line_width = 2, rgba = Rgba(0.0, 0.0, 0.0, 1))
plot_edge(s0,s1)

[[-0.          0.89920399]
 [ 0.          0.        ]
 [ 0.          0.        ]]


In [12]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, 
                                np.zeros(1),
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

Time to construct line certifier = 0.17500019073486328s


In [13]:

safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
print(safe)


INFO:drake:Bindings updated in 0.012 s
INFO:drake:Certification checked in  0.032 s


True


In [14]:
# tmp_map = line_certifier.get_polynomial_to_monomial_to_bindings_map()
# random.seed(0)
# p, (monom_to_bindings) = random.sample(tmp_map.items(), 1)[0]
# env = {}
# for i, s0_var in enumerate(line_certifier.get_s0()):
#     env[s0_var] = s0[i]
# for i, s1_var in enumerate(line_certifier.get_s1()):
#     env[s1_var] = s1[i]
# evaled_poly = p.EvaluatePartial(env)
# print(evaled_poly)
# for binding in monom_to_bindings.values():
#     print(binding)


In [15]:
for item in evaled_poly.items():
    print(item)

NameError: name 'evaled_poly' is not defined

In [ ]:
cspace_free_region_certifier = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# make the solver verbose
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

line_A = np.array([[-1],[1]])
line_b = np.array([s0.item(), s1.item()])
binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 0 
binary_search_options.epsilon_min = -1e-2
binary_search_options.max_iters = 15
# speed up the bisection search by taking non-uniform steps when possible
binary_search_options.search_d = True

In [ ]:
# construct the Lagrangian program to certify the box
alternation_tuples, d_minus_Ct, t_lower, \
t_upper, t_minus_t_lower, t_upper_minus_t, C, d, \
lagrangian_gram_vars, verified_gram_vars, separating_plane_vars, \
separating_plane_to_tuples, \
separating_plane_to_lorentz_cone_constraints = cspace_free_region_certifier.GenerateTuplesForBilinearAlternation(
    q_star, filtered_collision_pairs, line_A.shape[0]
)


prog = cspace_free_region_certifier.ConstructLagrangianProgram(alternation_tuples,
                                                               line_A,
                                                               line_b[:,np.newaxis],
                                                               lagrangian_gram_vars,
                                                               verified_gram_vars,
                                                               separating_plane_vars,
                                                               [],
                                                               np.array(t_lower)[:,np.newaxis],
                                                               np.array(t_upper)[:,np.newaxis],
                                                               VerificationOption(),
                                                               None
)

# certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(
#                                                                  q_star,
#                                                                  filtered_collision_pairs,
#                                                                  line_A,
#                                                                  line_b,
#                                                                  binary_search_options, 
#                                                                  solver_options)

In [ ]:
print(prog)

In [ ]:
rationals = cspace_free_region_certifier.GenerateRationalsForLinkOnOneSideOfPlane(q_star, filtered_collision_pairs)
for r in rationals:
    print(r.rational.numerator().decision_variables())
    print(r.a_A)
    print(r.b)
#     print(r.rational.denominator())
    print()

In [ ]:
certified_region_contraction_solution.Const